In [1]:
# https://nasa-openscapes.github.io/2021-Cloud-Hackathon/tutorials/02_Data_Discovery_CMR-STAC_API.html

from pystac_client import Client  
from collections import defaultdict    
import json
import geopandas
# import geoviews as gv
from cartopy import crs
# gv.extension('bokeh', 'matplotlib')
import geopandas as gpd

In [2]:
# find hls tiles given a point

def find_hls_tiles(point=False, band=False, limit=False, collections = ['HLSL30.v2.0', 'HLSS30.v2.0'], date_range = False):

    STAC_URL = 'https://cmr.earthdata.nasa.gov/stac'


    catalog = Client.open(f'{STAC_URL}/LPCLOUD/')



    try:
        x, y = point[0], point[1]
        # print(x,y)
    except TypeError:
        print("Point must be in the form of [lat,lon]")
        raise

    point = geopandas.points_from_xy([x],[y])
    point = point[0]



    # JOHN - THIS IS WHERE YOU WOULD ADD IN SEARCH PARAMETERS
    if date_range:

        search = catalog.search(
            collections=collections, intersects = point, datetime=date_range)
    else:
        search = catalog.search(
            collections=collections, intersects = point)



    # print(f'{search.matched()} Tiles Found...')


    item_collection = search.get_all_items()

    if limit:
        item_collection = item_collection[:limit]

    if band:
        links = []
        if type(band) == list:
            for i in item_collection:
                for b in band:
                    link = i.assets[b].href
                    # print(link)
                    links.append(link)
        
        else:
            for i in item_collection:
                link = i.assets[band].href
                links.append(link)
    
    else:
        links =[]
        for i in item_collection:
            # print(i.assets)
            for key in i.assets:
                if key.startswith('B'):
                    # link = i.assets[key].href.replace('https://data.lpdaac.earthdatacloud.nasa.gov/', 's3://')
                    link = i.assets[key].href

                    # print(link)
                    links.append(link)

    return links

In [3]:
# given a reach ID, find the nodes

import glob
import netCDF4
import os
import numpy as np


data_dir = '/home/confluence/data/mnt/input/sword'





def get_reach_nodes(data_dir, reach_id):

    all_nodes = []

    files = glob.glob(os.path.join(data_dir, '*'))
    print(f'Searching across {len(files)} continents for nodes...')

    for i in files:

        rootgrp = netCDF4.Dataset(i, "r", format="NETCDF4")

        node_ids_indexes = np.where(rootgrp.groups['nodes'].variables['reach_id'][:].data.astype('U') == str(reach_id))

        if len(node_ids_indexes[0])!=0:
            for y in node_ids_indexes[0]:
                node_id = str(rootgrp.groups['nodes'].variables['node_id'][y].data.astype('U'))
                all_nodes.append(node_id)



            # all_nodes.extend(node_ids[0].tolist())

        rootgrp.close()

    print(f'Found {len(set(all_nodes))} nodes...')
    return list(set(all_nodes))





# get_reach_nodes(data_dir,74270100221)

In [4]:
# given a reach ID, find the lat/lon points of all nodes



import glob
import netCDF4
import os
import numpy as np

data_dir = '/home/confluence/data/mnt/input/sword'


def get_reach_node_cords(data_dir, reach_id):

    all_nodes = []

    files = glob.glob(os.path.join(data_dir, '*'))
    print(f'Searching across {len(files)} continents for nodes...')

    for i in files:

        rootgrp = netCDF4.Dataset(i, "r", format="NETCDF4")

        node_ids_indexes = np.where(rootgrp.groups['nodes'].variables['reach_id'][:].data.astype('U') == str(reach_id))

        if len(node_ids_indexes[0])!=0:
            for y in node_ids_indexes[0]:

                lat = str(rootgrp.groups['nodes'].variables['x'][y].data.astype('U'))
                lon = str(rootgrp.groups['nodes'].variables['y'][y].data.astype('U'))
                all_nodes.append([lat,lon])



            # all_nodes.extend(node_ids[0].tolist())

        rootgrp.close()

    print(f'Found {len(all_nodes)} nodes...')
    return all_nodes











In [5]:
# given a reach ID, create download links for any hls tiles that intersect the nodes in the reach


def find_download_links_for_reach_tiles(data_dir, reach_id):
    node_coords = get_reach_node_cords(data_dir,reach_id)
    all_links = []
    for i in node_coords:
        print(i)
        links = find_hls_tiles(i,limit=1)
        print(links)
        all_links.extend(links)

    return list(set(all_links))

In [25]:

data_dir = '/home/confluence/data/mnt/input/sword'
reach_id = 74270100221

all_links =  find_download_links_for_reach_tiles(data_dir, reach_id)

Searching across 7 continents for nodes...
Found 84 nodes...
['-89.4447009885916', '37.4435521655434']


In [32]:
# https://nasa-openscapes.github.io/2021-Cloud-Hackathon/tutorials/05_Data_Access_Direct_S3.html

# need to make netrc file



# %matplotlib inline
import matplotlib.pyplot as plt
from datetime import datetime
import os
import requests
import boto3
import numpy as np
import xarray as xr
import rasterio as rio
from rasterio.session import AWSSession
from rasterio.plot import show
import rioxarray
# import geoviews as gv
# import hvplot.xarray
# import holoviews as hv
# gv.extension('bokeh', 'matplotlib')


s3_cred_endpoint = 'https://data.lpdaac.earthdatacloud.nasa.gov/s3credentials'

def get_temp_creds():
    temp_creds_url = s3_cred_endpoint
    return requests.get(temp_creds_url).json()


temp_creds_req = get_temp_creds()


session = boto3.Session(aws_access_key_id=temp_creds_req['accessKeyId'], 
                        aws_secret_access_key=temp_creds_req['secretAccessKey'],
                        aws_session_token=temp_creds_req['sessionToken'],
                        region_name='us-west-2')


rio_env = rio.Env(AWSSession(session),
                  GDAL_DISABLE_READDIR_ON_OPEN='EMPTY_DIR',
                  GDAL_HTTP_COOKIEFILE=os.path.expanduser('~/cookies.txt'),
                  GDAL_HTTP_COOKIEJAR=os.path.expanduser('~/cookies.txt'))

            
rio_env.__enter__()

In [33]:
hls_da = rioxarray.open_rasterio(list(all_links)[0], chuncks=True)

In [34]:

test = hls_da.values

KeyboardInterrupt: 

Exception ignored in: 'rasterio._env.log_error'
Traceback (most recent call last):
  File "/home/confluence/anaconda3/envs/hlstutorial/lib/python3.7/logging/__init__.py", line 1928, in getLogger
    def getLogger(name=None):
KeyboardInterrupt


In [ ]:
test

In [6]:
#generate

import pandas as pd


df = pd.read_csv('/data/data/downloads/Aqusat_TSS_v1.csv')

In [7]:
dates = df['date'];dates

0        2001-06-04
1        1992-06-02
2        2002-06-05
3        1992-08-19
4        1984-09-18
            ...    
32141    2008-08-11
32142    2017-10-16
32143    2013-09-27
32144    2016-07-18
32145    2010-10-13
Name: date, Length: 32146, dtype: object

In [8]:
date_range = "2001-06-04T00:00:00Z"

In [9]:
lat, lon, date = str(df.iloc[100]['lat']), str(df.iloc[100]['long']), df.iloc[0]['date']
i = [lat,lon]
# i = ['-89.4447009885916', '37.4435521655434']
# print(lat,lon,date)

In [10]:
for i in range(len(df)):
    lat, lon, date = str(df.iloc[i]['lat']), str(df.iloc[i]['long']), df.iloc[0]['date']
    point = [lat,lon]
    try:
        find_hls_tiles(point=[lat, lon], limit=1)
        print('success')
    except:
        pass

success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success


In [1]:
lat

NameError: name 'lat' is not defined